In [1]:
import configparser


def retrieve_openai_api_key() -> str:
    """
    function to parse the config file and retrieve OpenAI API key
    """

    config = configparser.ConfigParser()
    config.read('./../config.ini')
    api_key = config['API']['OpenAI_key']

    return api_key

api_key = retrieve_openai_api_key()

In [3]:
import sys
sys.path.append('../../')
from helper import read_in_data, add_textual_label


data_json = './../../data/task1_train.jsonl'
label_json = './../../data/sdg_label_mapping.json'
df = read_in_data(data_json)
df = add_textual_label(label_json, df)

# add a column containing concatenated title and abstract -> full_text
df['full_text'] = df.apply(lambda row: row['TITLE'] + ' ' + row['ABSTRACT'], axis=1)

df.head()

,ID,TITLE,ABSTRACT,URL,SDG,Label,full_text
0,oai:www.zora.uzh.ch:126666,Identifying phrasemes via interlingual associa...,"In corpus linguistics, statistical association...",https://www.zora.uzh.ch/id/eprint/126666,0,non-relevant,Identifying phrasemes via interlingual associa...
1,oai:www.zora.uzh.ch:106593,Synthesis of tripeptide derivatized cyclopenta...,We describe the syntheses of half-sandwich com...,https://www.zora.uzh.ch/id/eprint/106593,0,non-relevant,Synthesis of tripeptide derivatized cyclopenta...
2,oai:www.zora.uzh.ch:203310,Intelligence test items varying in capacity de...,There is a strong relationship between fluid i...,https://www.zora.uzh.ch/id/eprint/203310,0,non-relevant,Intelligence test items varying in capacity de...
3,oai:www.zora.uzh.ch:145601,Copy number increases of transposable elements...,Evolutionary dynamics of structural genetic va...,https://www.zora.uzh.ch/id/eprint/145601,14,Life Below Water,Copy number increases of transposable elements...
4,oai:www.zora.uzh.ch:188559,Linguistics : An Interdisciplinary Journal of ...,At the heart of this special issue are partiti...,https://www.zora.uzh.ch/id/eprint/188559,0,non-relevant,Linguistics : An Interdisciplinary Journal of ...


In [4]:
df_short = df[:5]

In [5]:
import json
import os
from openai import OpenAI
import sys



# from the bias project! use sparsely!
api_key = api_key
client = OpenAI(api_key=api_key)
model_name = "gpt-3.5-turbo"
temperature = 0
max_tokens = 500

for index, row in df_short.iterrows():
    text_to_classify = row['full_text']

    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "Classify by SDG (Sustainable Development Goal) number. Choose only the main SDG, one solution only. Respond only with the number of the goal: 'SDG < >'."},
            {"role": "system", "content": "Use 'SDG 0' if the text does not correspond to any of the SDGs and can be considered non-relevant."},           
            {"role": "user", "content": text_to_classify}
        ],
        temperature=temperature,
        max_tokens=max_tokens
        )


    if completion.choices:
        response = completion.choices[0].message
        print(f"Response: {response}")

Response: ChatCompletionMessage(content='SDG 4', role='assistant', function_call=None, tool_calls=None)
Response: ChatCompletionMessage(content='SDG 3', role='assistant', function_call=None, tool_calls=None)
Response: ChatCompletionMessage(content='SDG 4', role='assistant', function_call=None, tool_calls=None)
Response: ChatCompletionMessage(content='SDG 15', role='assistant', function_call=None, tool_calls=None)
Response: ChatCompletionMessage(content='SDG 4', role='assistant', function_call=None, tool_calls=None)
